In [1]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)
model.T = pe.Set(initialize = transshipment_points ,ordered = False)
model.C = pe.Set(initialize = customers ,ordered = False)
model.M = pe.Set(initialize = modes ,ordered = False)
model.P = pe.Set(initialize = periods, ordered = False)
model.W = pe.Set(initialize = warehouses, ordered = False)

# Decision Variables
# NOTE je kan ook lower en upper bounds geven aan de waarden
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # direct tonnes                         !!!TODO : meer uitleg !!!
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # indirect tonnes                       !!!TODO : meer uitleg !!!
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity block mode                   !!!TODO : meer uitleg !!!  
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity single wagon mode            !!!TODO : meer uitleg !!!
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity shortsea ship mode           !!!TODO : meer uitleg !!!
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity barge mode                   !!!TODO : meer uitleg !!!
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route     !!!TODO : meer uitleg !!!
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route   !!!TODO : meer uitleg !!!

#Additional variables 
model.td_w = pe.Var(model.F, model.W, model.P, domain = pe.NonNegativeIntegers)               
model.tid_w = pe.Var(model.T, model.W, model.P, domain = pe.NonNegativeIntegers)
model.x_w = pe.Var(model.F, model.W, model.P, domain = pe.NonNegativeReals)                   
model.y_w = pe.Var(model.T, model.W, model.P, domain = pe.NonNegativeReals)                  
model.w_c = pe.Var(model.W, model.C, model.P, domain = pe.NonNegativeReals)
model.inv_level = pe.Var(model.W, model.P, domain = pe.NonNegativeReals)

In [4]:
objExpr = sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C for p in model.P)     \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans for p in model.P)            \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans for p in model.P)         \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans for p in model.P)            \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans for p in model.P)                  \
        + sum(dist_point_cust[j, i] * transfer_cost * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)    \
        + sum(handling_costs[i] * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)                        \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C for p in model.P)                        \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C for p in model.P)                       \
        + sum(dist_fact_cust[i, j] * transfer_cost * model.x_w[i,j,p] for i in model.F for j in model.W for p in model.P)   \
        + sum(dist_point_cust[j, i] * transfer_cost * model.y_w[i,j,p] for i in model.T for j in model.W for p in model.P)  \
        + sum(model.td_w[i,j,p] * truck_fixed_cost for i in model.F for j in model.W for p in model.P)                      \
        + sum(model.tid_w[i,j,p] * truck_fixed_cost for i in model.T for j in model.W for p in model.P)                       
        
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [5]:
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

model.trucksDirectW = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.W:
            expression = model.td_w[i,j,p] >= model.x_w[i,j,p] / truck_cap
            model.trucksDirectW.add(expression)

model.trucksIndirectW = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.W:
            expression = model.tid_w[i,j,p] >= model.y_w[i,j,p] / truck_cap
            model.trucksIndirectW.add(expression)

model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.x_w[i,j,p] for j in model.W) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) + sum(model.w_c[i,j,p] for i in model.W if i == j) >= demands[p][j]
        model.meetDemand.add(expression)

model.noDelivery_w_c = pe.ConstraintList()
for p in model.P:
    for i in model.W:
        for j in model.C: 
            if i != j: 
                expression = model.w_c[i,j,p] == 0
                model.noDelivery_w_c.add(expression)

model.flowBalance = pe.ConstraintList()
for p in model.P:
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F) 
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F)
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
    for j in model.T: 
        expression = expression_bl + expression_s_w + expression_ss + expression_ba == sum(model.y[j,k,p] for k in model.C) + sum(model.y_w[j,w,p] for w in model.W)
        model.flowBalance.add(expression)

model.inventory = pe.ConstraintList()
for w in model.W:
    expression = model.inv_level[w,1] == InvLev[w] + sum(model.x_w[i,w,1] for i in model.F) + sum(model.y_w[i,w,1] for i in model.T) - sum(model.w_c[w,i,1] for i in model.C if i == w)
    model.inventory.add(expression)  

model.update_inventory = pe.ConstraintList()
for p in model.P:
    for w in model.W: 
        if p > 1:
            expression = model.inv_level[w,p] == model.inv_level[w,p-1] + sum(model.x_w[i,w,p] for i in model.F) + sum(model.y_w[i,w,p] for i in model.T) - sum(model.w_c[w,i,p] for i in model.C if i == w)
            model.update_inventory.add(expression)

In [6]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True, timelimit=5) # timelimit werkt wel maar vgm blijft ie op de achtergrond runnen

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-28
Read LP format model from file /var/folders/nh/v22djfjs2712hmlms5xl3z8h0000gs/T/tmpr8qsi9j2.pyomo.lp
Reading time = 0.03 seconds
x4191: 2421 rows, 4191 columns, 8383 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2421 rows, 4191 columns and 8383 nonzeros
Model fingerprint: 0xc08a6eb3
Variable types: 2591 continuous, 1600 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [2e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 3782825.6400
Presolve removed 2229 rows and 3409 columns
Presolve time: 0.01s
Presolved: 192 rows, 782 columns, 1564 nonzeros
Found heuristic solution: objective 3372366.3600
Variable types: 782 continuous, 0 integer (0 binary)

Root relaxation: 

In [ ]:
print("Objective value = "+ str(pe.value(model.obj)))